# Multiscale sediment dynamics

In [10]:
!pip install -r requirements.txt

In [11]:
%matplotlib notebook

from pathlib import Path
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import seaborn as sb

from common import sensitivity_tr_rbcd, plot_sensitivity_rbcds_transects
from sandpyper import ProfileDynamics

pd.options.mode.chained_assignment = None  # default='warn'

In [12]:
# set the test data folder path
test_data_folder = r"test_data"

ProfileSet_path=Path(test_data_folder + r"/test.p")

In [13]:
P=pickle.load(open(ProfileSet_path, "rb"))

In [14]:
labels=["Undefined", "Small", "Medium", "High", "Extreme"]

In [15]:
D = ProfileDynamics(P, bins=5, method="JenksCaspall", labels=labels)

Data will be partitioned into 5 discrete classes.
Labels provided.


In [16]:
D.compute_multitemporal(loc_full={'mar': 'Marengo',
         'leo': 'St. Leonards'}, filter_class='sand')

Filter activated: only ['sand'] points will be retained.
working on leo


  0%|          | 0/6 [00:00<?, ?it/s]

Calculating dt0, from 20180606 to 20180713 in leo.
Calculating dt1, from 20180713 to 20180920 in leo.
Calculating dt2, from 20180920 to 20190211 in leo.
Calculating dt3, from 20190211 to 20190328 in leo.
Calculating dt4, from 20190328 to 20190731 in leo.
working on mar


  0%|          | 0/1 [00:00<?, ?it/s]

done


TypeError: DataFrame.drop() takes from 1 to 2 positional arguments but 3 positional arguments (and 1 keyword-only argument) were given

In [ ]:
D.dh_df.head()

In [ ]:
D.dh_details

In [ ]:
D.lod_dh.head()

In [ ]:
D.lod_df.head()

In [ ]:
D.plot_lod_normality_check(locations=['leo'])

In [ ]:
# compute volumetrics

D.compute_volumetrics(lod=D.lod_df)

In [ ]:
# let's plot transect ID 10 in Marengo, for dt_2 and dt_3 with no classification applied.

D.plot_transects(location='mar', tr_id=10, from_date='20180727', to_date='20180925', classified=False)

In [ ]:
# same, with classification applied.

D.plot_transects(location='mar', tr_id=10, dt=['dt_2','dt_3'], classified=True)

In [ ]:
# plots mean elevation change of transect ID 10 in marengo across all time periods
# no LoD filter is applied, all the points are retained.

D.plot_transect_mecs(location='mar',tr_id=10)

In [ ]:
# same as above but with the LoD filter applied, therefore retaining only reliable elvation changes.

D.plot_transect_mecs(location='mar', lod=D.lod_df, tr_id=10)

In [ ]:
# LoD in each period of change in Marengo
D.lod_df.query("location=='mar'")[['dt','lod']]

In [ ]:
f, axs=plt.subplots(nrows=2,
                   ncols=2,
                   figsize=(8,8))

# Marengo dt_0
sb.kdeplot(data=D.dh_df.query("location=='mar' and dt=='dt_0'"),
            x='dh', ax=axs[0][0])
axs[0][0].axvspan(-0.03, 0.03, color='r', alpha=0.3, lw=0)
axs[0][0].set_title('Marengo dt_0')

# Marengo dt_2
sb.kdeplot(data=D.dh_df.query("location=='mar' and dt=='dt_2'"),
            x='dh', ax=axs[0][1])
axs[0][1].axvspan(-0.03, 0.03, color='r', alpha=0.3, lw=0)
axs[0][1].set_title('Marengo dt_2')

# Marengo dt_4
sb.kdeplot(data=D.dh_df.query("location=='mar' and dt=='dt_4'"),
            x='dh', ax=axs[1][0])
axs[1][0].axvspan(-0.1, 0.1, color='r', alpha=0.3, lw=0)
axs[1][0].set_title('Marengo dt_4')

# Marengo dt_6
sb.kdeplot(data=D.dh_df.query("location=='mar' and dt=='dt_6'"),
            x='dh', ax=axs[1][1])
axs[1][1].axvspan(-0.04, 0.04, color='r', alpha=0.3, lw=0)
axs[1][1].set_title('Marengo dt_6')

In [ ]:
D.plot_alongshore_change(mode='subset',       # to plot only a subset of locations and dts, rather than all
                         dt_subset=['dt_2', 'dt_3'],
                         location_subset=["mar"],
                         lod=D.lod_df, # LoD will be applied when provided in form of lod dataframe or single (global) value
                         ax2_y_lims=(-10, 20), # lineplot y axis limits
                        fig_size=(10,6),
                        font_scale=.9,
                        plots_spacing=0, # spacing between the two plots. Try .1, it is also nice
                        bottom=True,
                        y_heat_bottom_limit=40, # how far seawar the heatmap will extent.
                        heat_yticklabels_freq=10,
                        heat_xticklabels_freq=10,)

In [ ]:
D.plot_single_loc(["mar"],None)

In [ ]:
# scale_mode can be 'auto', which means the x axis is location-specific, or 'equal', better for comparison.

D.plot_mec_evolution(x_limits=(-.2, .8), scale_mode='equal',
                     loc_order=["leo","mar"],
                    x_binning=10,
                    figure_size=(6, 7),
                    font_scale=1.2)

In [ ]:
# scale_mode can be 'auto', which means the x axis is location-specific, or 'equal', better for comparison.

D.plot_mec_evolution(x_limits=(-.2, .8), scale_mode='auto',
                     loc_order=["leo","mar"],
                    x_binning=10,
                    figure_size=(6, 7),
                    font_scale=1.2)

In [ ]:
x_diff={'leo':[-0.12, 0.1]}

D.plot_mec_evolution(x_limits=(-.2, .8), scale_mode='equal',
                     x_diff=x_diff,
                     dates_step=15,
                     loc_order=["leo","mar"],
                    x_binning=10,
                    figure_size=(6, 7),
                    font_scale=1.2)

In [ ]:
D.LISA_site_level(mode="knn", k_value=50)

In [ ]:
D.hotspots.shape

In [ ]:
D.hotspots.query("location=='leo' and dt=='dt_2' and lisa_q in [1,3]").plot(column='lisa_q', categorical=True,
                                                                            cmap='RdBu')

In [ ]:
appendix=["_deposition", "_erosion"]

D.discretise(absolute=True, print_summary=True, lod=D.lod_df, appendix=appendix)

In [ ]:
D.infer_weights()

D.weights_dict

In [ ]:
# create the weight dictionary

my_weights_dict={'Undefined_deposition': 1,
 'Undefined_erosion': 1,
 'Small_deposition': 1,
 'Small_erosion': 1,
 'Medium_deposition': 1,
 'Medium_erosion': 1,
 'High_deposition': 3,
 'High_erosion': 3,
 'Extreme_deposition': 8,
 'Extreme_erosion': 8}

# add it to the ProfileDynamics object

D.weights_dict = my_weights_dict

D.weights_dict

In [ ]:
D.infer_weights()

D.weights_dict

In [ ]:
D.BCD_compute_location("geometry","all",True, filterit='lod')

In [ ]:
D.location_ebcds

In [ ]:
D.location_ss

In [ ]:
# the following dictionary is used to rename the magnitude of change classes into small codes
#such as 'me' for Medium Erosion or 'sd' for Small Deposition. This is done purely for graphical purposes.

relabel_dict={'Undefined_deposition': 'ue',
 'Undefined_erosion': 'ue',
 'Small_deposition': 'sd',
 'Small_erosion': 'se',
 'Medium_deposition': 'md',
 'Medium_erosion': 'me',
 'High_deposition': 'hd',
 'High_erosion': 'he',
 'Extreme_deposition': 'ed',
 'Extreme_erosion': 'ee'}

In [ ]:
%matplotlib inline

#let's plot those matrices

D.plot_trans_matrices(relabel_dict)

In [ ]:
%matplotlib inline

#let's plot those matrices

D.plot_trans_matrices(relabel_dict)

In [ ]:
order=[i for i in D.tags_order if i !='nnn']
ss=D.location_ss.loc[order] 
ss

In [ ]:
plt.rcParams['font.sans-serif'] = 'Arial'
plt.rcParams['font.family'] = 'sans-serif'
sb.set_context("paper", font_scale=2)


f,ax=plt.subplots(figsize=(12,10))

sb.heatmap(ss, cmap="Blues",annot=True,
           annot_kws={'size':14},linewidths=1,linecolor="white", cbar_kws={'label': 'Lim. Probabilities'});

#f.savefig(r'E:\\path\\to\\save\\picture.png', dpi=600); 

In [ ]:
loc_specs={'mar':{'thresh':6,
       'min_points':6}}

D.BCD_compute_transects(loc_specs=loc_specs,reliable_action='keep', dirNameTrans=D.ProfileSet.dirNameTrans)

In [ ]:
D.transects_rbcd.query("location=='mar'").plot(column="residual", cmap='RdBu_r')

In [ ]:
ss_tr_big = sensitivity_tr_rbcd(D,
                                test_thresholds='max',
                                test_min_pts=[0,20,2])

In [ ]:
f,ax=plt.subplots(figsize=(10,10))

palette=sb.color_palette( n_colors=ss_tr_big.query("location=='mar'").tr_id.unique().shape[0])
sb.lineplot(data=ss_tr_big.query("location=='mar'"), x='thresh',y='residual', hue='tr_id', 
            palette=palette, legend=False, **dict(alpha=0.5),
            ax=ax
)
ax.set_ylabel("r_bcd")
ax.axhline(y=0, lw=2, c='r');

In [ ]:
plot_sensitivity_rbcds_transects(ss_tr_big, location='mar')